In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
os.chdir('C:\Repos\DS-Apis-Globo')

In [3]:
file_inv = 'tvaberta_inventory_availability.csv(1).csv'
dateparser_inv = lambda x: datetime.strptime(x, '%d/%m/%Y')
sep_inv = ';'
parsedates_inv = ['date']

In [4]:
df_inv = pd.read_csv( file_inv,
                        sep = sep_inv,
                        parse_dates = parsedates_inv, 
                        date_parser = dateparser_inv)
df_inv.head()

,signal,program_code,date,available_time
0,SP1,HUCK,2020-07-25,10
1,SP1,HUCK,2020-08-01,300
2,SP1,HUCK,2020-08-08,118
3,SP1,HUCK,2020-08-15,106
4,SP1,HUCK,2020-08-22,6


In [5]:
df_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2292 entries, 0 to 2291
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   signal          2292 non-null   object        
 1   program_code    2292 non-null   object        
 2   date            2292 non-null   datetime64[ns]
 3   available_time  2292 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 71.8+ KB


In [6]:
file_pro = 'tvaberta_program_audience(1).csv'
dateparser_pro = lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
sep_pro = ','
parsedates_pro = ['program_start_time']
indexcol_pro = 'program_start_time'

In [7]:
df_pro = pd.read_csv( file_pro,
                        sep = sep_pro,
                        parse_dates = parsedates_pro, 
                        date_parser = dateparser_pro)
df_pro.head()

,signal,program_code,exhibition_date,program_start_time,average_audience
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05


In [8]:
df_pro['exhibition_date'] =  pd.to_datetime(df_pro['exhibition_date'], format='%Y-%m-%d')

In [69]:
df_res = df_pro.copy()
df_res = df_res.rename(columns={"exhibition_date": "date"})
df_res.rename(columns={"exhibition_date": "date"})
df_res['weekday'] = df_res.date.dt.dayofweek
df_res['predicted_audience'] = df_res['average_audience']
df_res['available_time'] = 0
df_res

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4,9.338920e+05,0
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2,1.835375e+06,0
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5,2.874812e+05,0
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6,8.649481e+04,0
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6,1.521656e+05,0
...,...,...,...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2020-05-04 21:31:00,2.093098e+07,0,2.093098e+07,0
8983,NAC,JHOJ,2020-05-13,2020-05-13 16:25:00,1.331286e+07,2,1.331286e+07,0
8984,NAC,SHOV,2020-06-07,2020-06-07 15:30:00,8.610939e+06,6,8.610939e+06,0
8985,NAC,SHOV,2020-05-17,2020-05-17 15:30:00,1.056899e+07,6,1.056899e+07,0


In [71]:
l_signal = df_res.signal.unique()
for v_signal in l_signal:
    #print(v_signal)
    df_aux = df_res.loc[df_res['signal'] == v_signal]   
    l_program_code = df_aux.program_code.unique()
    for v_program in l_program_code:
        #print("     " + v_signal)
        df_aux = df_res.loc[(df_res['signal'] == v_signal)&(df_res['program_code'] == v_program)] 
        l_weekdays = df_aux.weekday.unique()
        for v_day in l_weekdays:
            #print("     " + str(v_day))
            df_aux = df_res.loc[(df_aux['weekday'] == v_day)&(df_res['signal'] == v_signal)&(df_res['program_code'] == v_program)] 
            #print(v_signal,v_program,v_day)
            df_aux.sort_values(by=['average_audience'])
            df_aux['av2'] = np.roll(df_aux['average_audience'],2)
            df_aux['av3'] = np.roll(df_aux['average_audience'],3)
            df_aux['predicted_audience'] = df_aux.loc[:, ['av2', 'av3']].mean(axis=1)
            df_res.loc[df_aux.index] = df_aux

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\dldou\.conda\envs\datascience\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\dldou\.conda\envs\datascience\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

count     8987
unique       1
top       True
freq      8987
Name: predicted_audience, dtype: object

In [72]:
df_res

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4,1.070565e+06,0
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2,1.975505e+06,0
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5,3.371960e+05,0
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6,1.201617e+05,0
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6,1.333776e+05,0
...,...,...,...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2020-05-04 21:31:00,2.093098e+07,0,2.093098e+07,0
8983,NAC,JHOJ,2020-05-13,2020-05-13 16:25:00,1.331286e+07,2,1.331286e+07,0
8984,NAC,SHOV,2020-06-07,2020-06-07 15:30:00,8.610939e+06,6,9.232443e+06,0
8985,NAC,SHOV,2020-05-17,2020-05-17 15:30:00,1.056899e+07,6,8.947505e+06,0


In [78]:
df_res.date.describe()

C:\Users\dldou\.conda\envs\datascience\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                    8987
unique                     84
top       2020-05-22 00:00:00
freq                      132
first     2020-04-09 00:00:00
last      2020-07-01 00:00:00
Name: date, dtype: object

In [80]:
df_res = pd.concat([df_res,df_inv])
df_res

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4.0,1.070565e+06,0
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2.0,1.975505e+06,0
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5.0,3.371960e+05,0
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6.0,1.201617e+05,0
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6.0,1.333776e+05,0
...,...,...,...,...,...,...,...,...
2287,BH,NBRA,2020-08-24,NaT,NaN,NaN,NaN,218
2288,BH,NBRA,2020-08-25,NaT,NaN,NaN,NaN,66
2289,BH,NBRA,2020-08-26,NaT,NaN,NaN,NaN,44
2290,BH,NBRA,2020-08-27,NaT,NaN,NaN,NaN,190


In [81]:
df_res['weekday'] = df_res.date.dt.dayofweek
df_res

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4,1.070565e+06,0
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2,1.975505e+06,0
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5,3.371960e+05,0
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6,1.201617e+05,0
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6,1.333776e+05,0
...,...,...,...,...,...,...,...,...
2287,BH,NBRA,2020-08-24,NaT,NaN,0,NaN,218
2288,BH,NBRA,2020-08-25,NaT,NaN,1,NaN,66
2289,BH,NBRA,2020-08-26,NaT,NaN,2,NaN,44
2290,BH,NBRA,2020-08-27,NaT,NaN,3,NaN,190


In [82]:
del df_res['program_start_time']
del df_res['average_audience']
df_res

,signal,program_code,date,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,4,1.070565e+06,0
1,SP1,VALE,2020-06-24,2,1.975505e+06,0
2,BH,PTV1,2020-05-23,5,3.371960e+05,0
3,DF,AUTO,2020-05-03,6,1.201617e+05,0
4,DF,FGGE,2020-05-03,6,1.333776e+05,0
...,...,...,...,...,...,...
2287,BH,NBRA,2020-08-24,0,NaN,218
2288,BH,NBRA,2020-08-25,1,NaN,66
2289,BH,NBRA,2020-08-26,2,NaN,44
2290,BH,NBRA,2020-08-27,3,NaN,190
